In [6]:
import gist

from PIL import Image
import imageio
from IPython.display import display_png, clear_output

import sys
sys.path.append("../Library/")
import image_download as imd
import image_manipulation as ima

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from dotenv import load_dotenv
import os
load_dotenv("../.env")
import csv

import numpy as np
import db_connection as dbcon

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
from datetime import datetime

In [64]:
def get_filenames_unlabeled_images(filenames, labeled_images):
    unlabeled = list(set(filenames) - set(labeled_images))
    return unlabeled

def label_images(filenames, num_images):
    random.shuffle(filenames)
    
    labels = {}
    for i in range(num_images):
        imarray = np.asarray(Image.open(MFP_FOLDER + "processedIMG/" + filenames[i]))[:,:,:3]
        im = Image.fromarray(imarray)
        display_png(im)
        labels[filenames[i]] = input("Label:")
        clear_output()
        
    return labels
        

### Set Parameters

In [47]:
# parameters
BASE_FOLDER = os.getenv('USGS_FOLDER')
MFP_FOLDER = os.getenv('MFP_FOLDER')
MFP_FOLDER

'/Users/peterweber/Google Drive/MFP - Satellogic/'

### List all image filenames

In [99]:
filenames =  os.listdir(MFP_FOLDER + "processedIMG")
filenames[-10:]

['m_4209830_se_14_1_20140907_x4096_y1536.png',
 'm_4209830_se_14_1_20140907_x1536_y0.png',
 'm_3409456_ne_15_1_20170914_x4096_y5120.png',
 'm_4209725_sw_14_1_20140917_x5632_y4096.png',
 'm_2608164_sw_17_1_20171210_x6144_y6144.png',
 'm_2608164_sw_17_1_20171210_x6656_y3584.png',
 'm_3809356_nw_15_1_20140703_x6656_y1536.png',
 'm_3409456_ne_15_1_20170914_x6656_y3584.png',
 'm_3809356_nw_15_1_20140703_x6144_y512.png',
 'm_4209830_se_14_1_20140907_x1024_y4608.png']

### List images that are already labeled

In [100]:
labels_csv = os.listdir(MFP_FOLDER + "labels")

labeled_img = []
for fname_csv in labels_csv:
    df = pd.read_csv(MFP_FOLDER + "labels/" + fname_csv)
    for fname in df.filename:
        labeled_img.append(fname)

labeled_img[-10:]

['m_2608164_sw_17_1_20171210_x4096_y1536.png',
 'm_4210155_sw_14_1_20120710_x5632_y3072.png',
 'm_4709143_nw_15_1_20171010_x5120_y2560.png',
 'm_4209725_sw_14_1_20140917_x2048_y1536.png',
 'm_2608164_sw_17_1_20171210_x6656_y2048.png',
 'm_3909661_se_14_1_20170711_x3584_y512.png',
 'm_4209725_sw_14_1_20140917_x2048_y4608.png',
 'm_3812117_ne_10_1_20140726_x6144_y0.png',
 'm_3909661_se_14_1_20170711_x1024_y1024.png',
 'm_3812117_ne_10_1_20140726_x1536_y5120.png']

### Label images

In [101]:
unlabeled = get_filenames_unlabeled_images(filenames, labeled_img)
labels = label_images(filenames, 20)

df_labels = pd.DataFrame({
        'filename': list(labels.keys()),
        'label': list(labels.values())
})
df_labels.tail()

,filename,label
15,m_3812117_ne_10_1_20140726_x0_y0.png,1
16,m_4209725_sw_14_1_20140917_x4096_y4096.png,1
17,m_4209830_se_14_1_20140907_x2560_y4096.png,1
18,m_4209725_sw_14_1_20140917_x6656_y3072.png,1
19,m_2608164_sw_17_1_20171210_x0_y0.png,0


### Save as .csv

In [102]:
df_labels.to_csv(MFP_FOLDER + "/labels/labels_" + str(datetime.now()) + ".csv")